In [1]:
# from Bio import Entrez
# import time
import os
import csv
from xlsxwriter.workbook import Workbook
import pandas as pd

## FILTRAR GENES RIKEN Y PASAR A EXTENSIÓN EXCEL

In [2]:
group1 = "male"
group2 = "female"
genotype = "GTEx_Liver"

dir = f"/data/05_deseq/{genotype}"
file = f"deseq_all_{group1}_vs_{group2}.tsv"
out = f"deseq_all_{group1}_vs_{group2}_noRIKEN.tsv"

lines = []
with open(f'{os.getcwd()}{dir}/{file}', 'r') as deseq:
    for line in deseq.readlines():
        if not any(x in line.split('\t')[-1] for x in ['RIKEN', 'Riken']):
            lines.append(line)

n_line = 0
with open(f'{os.getcwd()}{dir}/{out}', 'w') as out:
    for line in lines:
        out.write(line)
        print(f'Writing line {n_line}', end='\r')
        n_line += 1

In [3]:
dir = f"/data/05_deseq/{genotype}"
# Input file path
tsv_file = f'deseq_all_{group1}_vs_{group2}_noRIKEN.tsv'
# Output file path
xlsx_file = f'deseq_all_{group1}_vs_{group2}_noRIKEN.xlsx'

# Creating an XlsxWriter workbook object and adding 
# a worksheet.
workbook = Workbook(f'{os.getcwd()}{dir}/{xlsx_file}')
worksheet = workbook.add_worksheet()

# Reading the tsv file.
read_tsv = csv.reader(open(f'{os.getcwd()}{dir}/{tsv_file}', 'r', encoding='utf-8'), delimiter='\t')

# We'll use a loop with enumerate to pass the data 
# together with its row position number, which we'll
# use as the cell number in the write_row() function.
for row, data in enumerate(read_tsv):
    worksheet.write_row(row, 0, data)

# Closing the xlsx file.
workbook.close()

## NO SHRINKAGE VS SHRINKAGE

In [ ]:
group1 = 3
group2 = 4

dir = "/data/05_deseq/SKA113"
f_unshrunken = f"deseq_all_G{group1}_vs_G{group2}.tsv"
f_shrunken = f"deseq_all_G{group1}_vs_G{group2}_shrunken.tsv"

df_unshrunken = pd.read_table(f'{os.getcwd()}{dir}/{f_unshrunken}')
df_shrunken = pd.read_table(f'{os.getcwd()}{dir}/{f_shrunken}')

In [ ]:
df_unshrunken.head()

,symbol,baseMean,FoldChange,log2FoldChange,lfcSE,stat,pvalue,padj,description
0,0610005C13Rik,6908.323167,0.971525,-0.041677,0.154546,-0.269670,0.787414,0.918927,RIKEN cDNA 0610005C13 gene
1,1700018A23Rik,32.251024,1.055603,0.078067,0.579951,0.134610,0.892921,0.968568,RIKEN cDNA 1700018A23 gene
2,1700018L02Rik,94.490703,1.045674,0.064433,0.554774,0.116143,0.907539,0.973200,RIKEN cDNA 1700018L02 gene
3,1700066M21Rik,1249.238694,1.734989,0.794926,0.200671,3.961332,0.000075,0.003471,RIKEN cDNA 1700066M21 gene
4,1700086O06Rik,147.976671,0.845058,-0.242878,0.334435,-0.726234,0.467696,0.761302,RIKEN cDNA 1700086O06 gene


In [59]:
if list(df_unshrunken.loc[:, 'symbol']) == list(df_shrunken.loc[:, 'symbol']):
    print('Both unshrunken and shrunken deseq2 results contain the same list of genes')
else:
    print('Unshrunken and shrunken deseq2 results differ in their list of genes')

unshrunken_list = df_unshrunken.loc[:, ['symbol', 'log2FoldChange']]
shrunken_list = df_shrunken.loc[:, ['symbol', 'log2FoldChange']]

unshrunken_list.set_index('symbol', inplace=True)
shrunken_list.set_index('symbol', inplace=True)
df_comb = unshrunken_list.join(shrunken_list, lsuffix='_unshrunken', rsuffix='_shrunken', how='outer').reset_index()
df_comb

Both unshrunken and shrunken deseq2 results contain the same list of genes


,symbol,log2FoldChange_unshrunken,log2FoldChange_shrunken
0,0610005C13Rik,-0.041677,-0.022558
1,1700018A23Rik,0.078067,0.006216
2,1700018L02Rik,0.064433,0.005478
3,1700066M21Rik,0.794926,0.690778
4,1700086O06Rik,-0.242878,-0.050348
...,...,...,...
1368,Yipf2,-0.147817,-0.069755
1369,Yme1l1,0.180473,0.106559
1370,Yrdc,-0.057540,-0.017899
1371,Zcchc18,1.326703,0.022063


In [60]:
df_comb.loc[:, 'log2FoldChange_unshrunken'] - df_comb.loc[:, 'log2FoldChange_shrunken']

0      -0.019119
1       0.071851
2       0.058955
3       0.104149
4      -0.192530
          ...   
1368   -0.078061
1369    0.073914
1370   -0.039640
1371    1.304640
1372    0.150416
Length: 1373, dtype: float64